In [2]:
import rdflib
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [ ]:

filename = "../WebProtege/digitrubber-full.ttl"


g = rdflib.Graph()

g.parse(filename, format='ttl')

query = """

SELECT ?Class ?Label ?Definition
WHERE {
    ?Class <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#Class> ;
           <http://www.w3.org/2000/01/rdf-schema#label> ?Label ;
           <http://purl.obolibrary.org/obo/IAO_0000115> ?Definition.
    Filter(lang(?Definition) !='de')
    Filter(lang(?Label) !='de')
}

"""

qres = g.query(query)

clss_def_dict = {'Class':[], 'Label':[], 'Definition':[]}

for row in qres:
    clss_def_dict['Class'].append(row.asdict()['Class'].split('/')[-1])
    clss_def_dict['Label'].append(row.asdict()['Label'])
    clss_def_dict['Definition'].append(row.asdict()['Definition'])
    # print(row.asdict()['Class'], row.asdict()['Label'] row.asdict()['Definition'])
    # print("%s \t %s " % row)
    # print("%s" % row)

df = pd.DataFrame(data=clss_def_dict )

df.to_csv('class_definition.csv', encoding='utf-8')

In [ ]:
from sentence_transformers import SentenceTransformer, util

df = pd.read_csv('class_definition.csv', encoding='utf-8')

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

#Compute embedding for both lists
emb_mat = [model.encode(definition, convert_to_tensor=False) for definition in df['Definition'].values]
df['embedding'] = emb_mat


In [6]:
# compute the cosine similarity

def cosine_sim(i,j):
    return dot(i, j)/(norm(i)*norm(j))
    

emb_sim_mat = np.array([[cosine_sim(i, j) for j in df['embedding'].values] for i in df['embedding'].values])

emb_sim_mat_bool = emb_sim_mat > 0.9

labels = df['Label'].values

In [17]:
import plotly.express as px


fig = px.imshow(emb_sim_mat_bool, x=labels, y=labels, color_continuous_scale='Viridis', aspect="auto", width=3000, height=3000)
fig.update_xaxes(side="top")
fig.write_html('term_similarity.html', auto_open=True)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
import plotly.express as px


fig = px.imshow(emb_sim_mat_bool, x=labels, y=labels, color_continuous_scale='blues', aspect="auto", width=1000, height=1000, text_auto=False)
fig.update_xaxes(side="top")
fig.write_html('term_similarity_blue.html', auto_open=True)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [57]:
df_emb = pd.DataFrame(emb_mat)

df_emb.to_csv('emb_vector.tsv', sep='\t', index=False)

df[['Label', 'Definition']].to_csv('metadata.tsv', sep='\t', index=False)